In [2]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
import functions_forcrawl
edit = functions_forcrawl.All_func()

def clean_semicolon(st):
    
    if re.search(';$|; $', st, flags=re.S):
        return re.sub(';$|; $', '', st, flags=re.S).strip()
    
    else:
        return st
    
def add_minutes(t, m):
    
    hours, minutes = map(int, t.split(":"))
    total_minutes = 60 * hours + minutes + m
    new_hours = total_minutes // 60
    new_minutes = total_minutes % 60
    return "{:02d}:{:02d}".format(new_hours % 24, new_minutes)


cookies = {
    'PHPSESSID': 'lbwww6~2f22d77a3de3da12104acedee740dc52',
    'joe-chnlcustid': '0a43eed3-8ddd-4ab1-a6c7-c5256b4d9c49',
}

headers = {
    'Accept': 'text/html, */*; q=0.01',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'Cookie': 'PHPSESSID=lbwww6~2f22d77a3de3da12104acedee740dc52; joe-chnlcustid=0a43eed3-8ddd-4ab1-a6c7-c5256b4d9c49',
    'Origin': 'https://online.eccmid.org',
    'Referer': 'https://online.eccmid.org/programme-live-1?coday=2023-02-16&embed=',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

data = {
    'idCat': '1',
    'sessionRef': '3379',
    'timezone': 'Europe/Paris',
    'defaultTimezone': 'Europe/Paris',
    'dispCountry': '',
    'target': '_self',
    'embed': 'false',
    'dispCities': '',
    'firstnameFull': '',
    'page': '',
}

response = requests.post(
    'https://online.eccmid.org/fo-user-display-session-details.php',
    cookies=cookies,
    headers=headers,
    data=data,
)

dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_title':[],
    'session_number':[],
    'name':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

dates = ['02-15', '02-16', '03-15', '03-16', '04-15', '04-16', '04-17', '04-18', '05-24', '05-25']


In [3]:
se_no = 1
for dat in dates:
    
    reqout = requests.get(f'https://eccmid2023.key4.live/programme-live-1?coday=2023-{dat}&embed=1')
    soup = BS(reqout.content, 'lxml')
    
    outer_row = soup.find_all(class_='row live-program-card-row-grid')
    locations = soup.find_all(class_='d-flex-block text-center roomsName py-2')
    
    for ou in outer_row:

        col_div = 0
        
        for i_id in ou.find_all(class_='border text-center program-grid-calandar'):
            
            if i_id.find(class_='programme-session-card programme-session-display border'):
                
                session_number = f'S{se_no}'
                print(f'=====**====={session_number}=====**=======', col_div)
                i_id = i_id.find(class_='programme-session-card programme-session-display border')

                data['sessionRef'] = i_id.get('data-session-reference')
                
                req = requests.post(
                    'https://eccmid2023.key4.live/fo-user-display-session-details.php',
                    cookies=cookies,
                    headers=headers,
                    data=data,)

                soupin = BS(req.content, 'lxml')
                sess_data = soupin.find(id='modal-session-content')

                # -------------------------------------    date time location ya type     ------------------------------------
                sess_date = ''
                sess_time = ''
                sess_loc = ''
                sess_type = ''
                sess_loc = ''
                sess_loc = locations[col_div].text.strip()
                date_time_type = edit.text_cleaner(sess_data.find(class_='modal-session-header').text)

                sess_date = date_time_type.split(',')[0].strip()
                sess_time = date_time_type.split(',')[1].split('CET')[0].strip()
                sess_type = date_time_type.split('CET')[-1].strip()

                # --------------------------------------  category or type and title -----------------------------------------------------
                sess_cate = ''
                sess_cate = sess_data.find(class_='modal-title-cat mt-2 mb-2').text.strip()
                sess_title = sess_data.find(class_='modal-session-name mb-2').text.strip()
                sess_title = i_id.get('data-session-reference') + ' - ' + sess_title

                # --------------------------------------- abstract text OR co organised ---------------------------------------------
                co_org = ''
                sess_text = ''
                if sess_data.find(class_='modal-session-organized-by mt-2'):
                    co_org = sess_data.find(class_='modal-session-organized-by mt-2').text.strip()
                if sess_data.find(class_='modal-session-desc mt-2'):
                    sess_text = sess_data.find(class_='modal-session-desc mt-2').text.strip()
                    
                sess_text = co_org + ' ' + sess_text

                # ------------------------------------------- session ka authors aff --------------------------------------------------
                sess_author = ''
                sess_aff = ''

                if sess_data.find(class_='modal-session-moderators'):
                    for i in sess_data.find(class_='modal-session-moderators').find_all(class_='row'):

                        aff = i.find(class_='modal-session-moderator-country').text.strip()
                        auth = i.text.strip()
                        auth = auth.replace(aff, '').split('-')[0]
                        aff = aff.replace(',', '').strip()

                        sess_author += auth + '; '
                        sess_aff += aff + '; '

                sess_author = clean_semicolon(sess_author)
                sess_aff = clean_semicolon(sess_aff)
                sess_date = sess_date + '/2023'
                sess_type = re.sub(sess_loc, '', sess_type)

                print(f'sess date :- {sess_date}')
                print(f'sess time :- {sess_time}')
                print(f'sess loca :- {sess_loc}')
                print(f'sess type :- {sess_type}')
                print(f'sess cate :- {sess_cate}')
                print(f'sess title :- {sess_title}')
                print(f'abs text :- {sess_text}')
                print(f'sess auth :- {sess_author}')
                print(f'sess aff :- {sess_aff}')

                dic['session_number'].append(session_number)
                dic['name'].append('Session')
                dic['source_id'].append('')
                dic['manual_id'].append('')
                dic['article_title'].append(sess_title)
                dic['url'].append(f'https://eccmid2023.key4.live/programme-live-1?coday=2023-{dat}&embed=1')
                dic['authors'].append(sess_author)
                dic['author_affiliation'].append(sess_aff)
                dic['abstract_text'].append(sess_text)
                dic['date'].append(sess_date)
                dic['start_time'].append(sess_time.split('-')[0].strip())
                dic['end_time'].append(sess_time.split('-')[-1].strip())
                dic['location'].append(sess_loc)
                dic['session_title'].append(sess_title)
                dic['session_type'].append(sess_type)
                dic['category'].append(sess_cate)
                dic['sub_category'].append('')
                dic['disclosure'].append('')
                print('======'*20)
                
                # *+*+*+*+*+*+*+*+*+*+*+*+*  ------------------- ABSTRACT DATA ----------------- *+*+*+*+*+*+**+**+*+*+*
                
                abs_data = sess_data.find(class_='modal-sessions-interventions')
                if abs_data:
                    
                    pno = 1
                    for ab in abs_data.find_all(class_='modal-sessions-interventions-group clearfix'):

                        abs_title = ab.find(style='font-weight: bold').text.strip()

                        # abs author
                        abs_author = ''
                        abs_aff = ''
                        if ab.find(class_='modal-session-faculties mb-1'):
                            ab_authaff = ab.find(class_='modal-session-faculties mb-1').text.strip()
                            abs_author = ', '.join(ab_authaff.split(',')[:-1])
                            abs_aff = ab_authaff.split(',')[-1]

                        if abs_title == '':
                            abs_title = 'TBD'
                            
                        # source id...... time shit
                        
                        ar_id = ''
                        art_time = ''
                        
                        if re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title):
                            src_code = re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title).group()

                            if src_code:

                                if re.search('^\d{1,6}', src_code):
                                    ar_id = src_code + '-' + str(pno)
                                    art_time = ''
                                elif re.search('^[A-Z]{1,5}\d{1,10}', src_code):

                                    str_pno = str(pno)
                                    if len(str_pno) == 1:
                                        str_pno = '0' + str_pno
                                    ar_id = src_code + str_pno
                                    art_time = sess_time
                        else:
                            ar_id = 'QC'
                        
                            
                        print(ar_id, '===', art_time)
                        print(f'abs title :- {abs_title}')
                        print(f'abs auth :- {abs_author}')
                        print(f'abs aff :- {abs_aff}')

                        dic['session_number'].append(session_number)
                        dic['name'].append('Abstract')
                        dic['source_id'].append(ar_id)
                        dic['manual_id'].append('')
                        dic['article_title'].append(abs_title)
                        dic['url'].append(f'https://eccmid2023.key4.live/programme-live-1?coday=2023-{dat}&embed=1')
                        dic['authors'].append(abs_author)
                        dic['author_affiliation'].append(abs_aff)
                        dic['abstract_text'].append('')
                        dic['date'].append(sess_date)
                        dic['start_time'].append('')
                        dic['end_time'].append('')
                        dic['location'].append(sess_loc)
                        dic['session_title'].append(sess_title)
                        dic['session_type'].append(sess_type)
                        dic['category'].append(sess_cate)
                        dic['sub_category'].append('')
                        dic['disclosure'].append('')
                        print('-*-+-'* 17)
                        pno += 1
                        
                se_no += 1
                print('inr', se_no)
                
            col_div += 1
            

=====**=====S1=====**======= 0
sess date :- 15/02/2023
sess time :- 16:00 - 17:00
sess loca :- Science track
sess type :- 1-hour SymposiumScience track
sess cate :- 7. Parasitic diseases & global health
sess title :- 3437 - Multi-country outbreaks: from signal to response
abs text :- Co-organised with                    
Eurosurveillance, European Centre for Disease Prevention and Control (ECDC) During the first half of 2022, several viral and bacterial multi-country outbreaks have illustrated challenges in the detection and response to such events. In this session, speakers will present two outbreaks in Europe, and cover signal detection, outbreak investigation and response measures and follow up. They will demonstrate the importance of cross-disciplinary collaboration that involves clinicians, epidemiologists, microbiologists as well as international coordinating bodies such as the ECDC, EFSA and WHO to gather sound evidence to guide public health decision making in complex situation

sess date :- 16/02/2023
sess time :- 18:00 - 19:00
sess loca :- Science track
sess type :- 1-hour SymposiumScience track
sess cate :- 1. Viral infection & disease
sess title :- 3425 - Emerging and re-emerging haemorrhagic fever viruses
abs text :- Co-organised with                    
Pan-American Association Infectious Diseases (API), ESCMID Study Group for Veterinary Microbiology (ESGVM), ESCMID Study Group for Infections in Travellers and Migrants (ESGITM) Viral haemorrhagic fever (VHF) is a group of acute zoonotic diseases with high mortality rates caused by seven families of viruses that can infect both humans and animals. Diseases caused by haemorragic fever viruses are endemic in certain parts of the world and can cause major outbreaks. Emerging and re-emerging VHFs remain a great health concern across the world due to poor prognosis and lack of specific vaccines or drugs for effective treatment. This session provides two examples, from different geographical regions.
sess auth 

sess date :- 16/03/2023
sess time :- 17:00 - 18:00
sess loca :- Science track
sess type :- Meet-the-ExpertScience track
sess cate :- 5. New antibacterial agents, PK/PD & Stewardship
sess title :- 3180 - Harmonisation of phage susceptibility testing
abs text :- Co-organised with                    
ESCMID Study Group for Non-traditional Antibacterial Therapy (ESGNTA) Phages are currently being used in compassionate cases to treat persistent and resistant bacterial infections throughout the world. Additionally, many clinical studies are currently emerging to scientifically test the value and role of phages in clinical use. As phages have a narrow spectrum of activity, sensitivity testing of the patient strain is a prerequisite to treatment. While conventional laboratory methods exist to this end, no standardized and validated methods for phage sensitivity currently exist. In this session we will explore the current knowledge and future directions of phage susceptibility testing that will

sess date :- 15/04/2023
sess time :- 08:30 - 10:30
sess loca :- Hall D
sess type :- 2-hour SymposiumHall D
sess cate :- 4. Diagnostic microbiology
sess title :- 3289 - Syndromic testing for infectious disease in LMICs settings: challenges and opportunities
abs text :- Co-organised with                    
ESCMID International Affairs Subcommittee (EIAS), ESCMID Study Group for Genomic and Molecular Diagnostics (ESGMD) In the COVID-19 pandemic era, low- middle-income countries (LMICs) made a great effort to implement molecular diagnostic laboratories to support test-trace-isolate (TTI) strategies. The increased diagnostic capabilities have facilitated a new horizon for using molecular diagnostic techniques for infectious diseases in these countries, thereby improving diagnostic capabilities. Indeed, syndromic diagnostic techniques for respiratory and gastrointestinal infections are commonly used in developed countries. These techniques offer many advantages for the clinical management o

sess date :- 15/04/2023
sess time :- 08:30 - 10:30
sess loca :- Hall J
sess type :- 2-hour SymposiumHall J
sess cate :- 5. New antibacterial agents, PK/PD & Stewardship
sess title :- 3369 - Antibiotic stewardship for urinary tract infections: challenges and opportunities
abs text :- Co-organised with                    
CMI Board of Editors Urinary tract infections are among the most common indications for antibiotic treatment across all care settings (community and facility). Improved antimicrobial stewardship for urinary tract infections would, therefore, potentially substantially impact better antibiotic use. The symposium will allow the audience to understand in-depth the challenges and opportunities to adopt antibiotic stewardship approaches in managing urinary tract infections.
sess auth :- Candice BONACONSA ; Scott HULTGREN
sess aff :- South Africa; United States
3369-1 === 
abs title :- Do we all mean the same when we say "complicated urinary tract infection"?
abs auth :- 
abs 

...

sess date :- 18/04/2023
sess time :- 13:30 - 15:30
sess loca :- Hall E
sess type :- 2-hour Oral SessionHall E
sess cate :- 7. Parasitic diseases & global health
sess title :- PAK15 - Malaria: challenges in LMICs and HICs
abs text :-  
sess auth :- 
sess aff :- 
PAK1501 === 13:30 - 15:30
abs title :- Side effects and factors influencing non-adherence to mass drug administration for malaria control in the context of a cluster randomised clinical trial (MATAMAL) on the Bijagós Archipelago, Guinea-Bissau
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
PAK1502 === 13:30 - 15:30
abs title :- Non-typhoidal Salmonella co-infect children with malaria and impair outcome of hospital-admitted children under five in DR Congo
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
PAK1503 === 13:30 - 15:30
abs title :- Impact of the COVID-19 pandemic on maternal and malaria services 

sess date :- 18/04/2023
sess time :- 13:30 - 15:30
sess loca :- Hall J
sess type :- 2-hour SymposiumHall J
sess cate :- 10. Immunology & vaccinology
sess title :- 3190 - Changing the paradigm of neutropaenic fever prevention and management
abs text :- Co-organised with                    
ESCMID Study Group for Antimicrobial Stewardship (ESGAP), ESCMID Study Group for Infections in Compromised Hosts (ESGICH) There is emerging evidence that antimicrobial use in cancer patients can impact treatment response. With novel drugs and therapies, including immunotherapeutics and CAR-T cells, the clinical epidemiology of neutropenic fever has changed. Therefore, treatment pathways should reflect optimal care of these patients, minimising their antimicrobial exposures and the associated unintended consequences, including dysbiosis, antimicrobial resistance, and prolonged hospital stays. This session will bring experts from the field to discuss novel approaches to management of the most common inf

sess date :- 18/04/2023
sess time :- 13:30 - 14:30
sess loca :- Arena 1
sess type :- 1-hour ePoster Flash SessionArena 1
sess cate :- 6. Fungal infection & disease
sess title :- MEM03 - What's new in the fungal diagnostic field
abs text :-  
sess auth :- Jon SALMANTON
sess aff :- Germany
MEM0301 === 13:30 - 14:30
abs title :- The current state of laboratory mycology in Asia/Pacific: a survey from the European Confederation of Medical Mycology (ECMM) and International Society for Human and Animal Mycology (ISHAM)
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
MEM0302 === 13:30 - 14:30
abs title :- Development and performance of MycoMEIA – Aspergillus: a new urine diagnostic for aspergillosis
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
MEM0303 === 13:30 - 14:30
abs title :- Does (1,3)-β-d-glucan predict the degree of pulmonary involvement in Pneumocystis jiro

sess date :- 18/04/2023
sess time :- 14:30 - 15:30
sess loca :- Arena 4
sess type :- 1-hour ePoster Flash SessionArena 4
sess cate :- 10. Immunology & vaccinology
sess title :- CSP01 - Immune response in vaccine preventable diseases
abs text :-  
sess auth :- 
sess aff :- 
CSP0101 === 14:30 - 15:30
abs title :- Estimating the Health and Economic Burden of Pneumococcal diseases Attributable to V116 Vs. PCV20 Serotypes Among Adults in Italy
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
CSP0102 === 14:30 - 15:30
abs title :- Adaptive immune responses to aerosolised bacillus Calmette-Guérin (BCG) as a surrogate controlled human infection model for Mycobacterium tuberculosis
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
CSP0103 === 14:30 - 15:30
abs title :- Use of MVA-BN vaccine in the context of the current mpox outbreak: early experience of an infectious and t

sess date :- 25/05/2023
sess time :- 17:00 - 18:00
sess loca :- Science track
sess type :- 1-hour SymposiumScience track
sess cate :- 5. New antibacterial agents, PK/PD & Stewardship
sess title :- 3440 - Rapid diagnostics to inform the management of acute respiratory
abs text :- Co-organised with                    
Value-Dx Acute respiratory tract infections are among the most frequently encountered infectious diseases problems in both community and hospital settings. These infections are also the leading cause of unnecessary use of empirical antimicrobials and one of main drivers of the emergence of antibiotic resistance. Availability and rationale use of reliable point-of-care tests and various rapid diagnostic tools in this setting may improve antimicrobial usage for these infections. VALUE-Dx is an IMI project and partnered by ESCMID and European Respiratory Society (ERS) for studying this issue and a consensus report in underway by experts involved from both Societies and by thos

In [4]:
df = pd.DataFrame(dic)
df

,source_id,manual_id,article_title,url,authors,author_affiliation,abstract_text,date,start_time,end_time,location,session_title,session_number,name,session_type,category,sub_category,disclosure
0,,,3437 - Multi-country outbreaks: from signal to...,https://eccmid2023.key4.live/programme-live-1?...,Jacob MORAN; Ines STEFFENS,Israel; Sweden,Co-organised with \nEurosur...,15/02/2023,16:00,17:00,Science track,3437 - Multi-country outbreaks: from signal to...,S1,Session,1-hour SymposiumScience track,7. Parasitic diseases & global health,,
1,3437-1,,A dangerous pre-Easter surprise: multi-country...,https://eccmid2023.key4.live/programme-live-1?...,,,,15/02/2023,,,Science track,3437 - Multi-country outbreaks: from signal to...,S1,Presentation,1-hour SymposiumScience track,7. Parasitic diseases & global health,,
2,3437-2,,Multi-country detections of diphtheria in migr...,https://eccmid2023.key4.live/programme-live-1?...,,,,15/02/2023,,,Science track,3437 - Multi-country outbreaks: from signal to...,S1,Presentation,1-hour SymposiumScience track,7. Parasitic diseases & global health,,
3,,,"3268 - Tick-borne diseases in Europe: new, eme...",https://eccmid2023.key4.live/programme-live-1?...,Nanna Skaarup ANDERSEN ; Olli VAPALAHTI,Denmark; Finland,Co-organised with \nDanish ...,15/02/2023,17:00,18:00,Science track,"3268 - Tick-borne diseases in Europe: new, eme...",S2,Session,1-hour SymposiumScience track,2. Bacterial infection & disease,,
4,3268-1,,Lyme borreliosis: recent progress and remainin...,https://eccmid2023.key4.live/programme-live-1?...,,,,15/02/2023,,,Science track,"3268 - Tick-borne diseases in Europe: new, eme...",S2,Presentation,1-hour SymposiumScience track,2. Bacterial infection & disease,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,3440-1,,Use of rapid diagnostics in ARTIs: a consensus...,https://eccmid2023.key4.live/programme-live-1?...,,,,25/05/2023,,,Science track,3440 - Rapid diagnostics to inform the managem...,S381,Presentation,1-hour SymposiumScience track,"5. New antibacterial agents, PK/PD & Stewardship",,
1871,3440-2,,Implementation of the consensus report in the ...,https://eccmid2023.key4.live/programme-live-1?...,,,,25/05/2023,,,Science track,3440 - Rapid diagnostics to inform the managem...,S381,Presentation,1-hour SymposiumScience track,"5. New antibacterial agents, PK/PD & Stewardship",,
1872,,,3314 - New developments in clinical radiology ...,https://eccmid2023.key4.live/programme-live-1?...,,,Co-organised with \nESCMID ...,25/05/2023,18:00,19:00,Science track,3314 - New developments in clinical radiology ...,S382,Session,Meet-the-ExpertScience track,2. Bacterial infection & disease,,
1873,3314-1,,TBD,https://eccmid2023.key4.live/programme-live-1?...,,,,25/05/2023,,,Science track,3314 - New developments in clinical radiology ...,S382,Presentation,Meet-the-ExpertScience track,2. Bacterial infection & disease,,


In [20]:
df.to_excel('binaFilter_all.xlsx', index=False)